In [12]:
import numpy as np
import pandas as pd
import time

In [13]:
def simulation(nrespondents,k,samplescheme,batchsize,itemfile,designfile,mis=0,seed=0):
    if seed>0:
        np.random.seed(seed)#for reproducibility
    if mis>0:
        data1=pd.read_csv('HB_120items.csv')
        utilsall=data1.get_values()[:,2:]
        realmean=np.mean(utilsall,0)
        topmis=np.argsort(realmean)[::-1][:mis]
    data=pd.read_csv(itemfile)#
    MDDesign =pd.read_csv(designfile)#get the files
    utilsall=data.get_values()[:,2:]#utils of bots
    [numreal,nitems]=utilsall.shape#
    design=MDDesign.get_values()[:,2:]#changes to a numpy array
    numver=max(MDDesign['Version'])#number of versions
    qperset=max(MDDesign['Set'])#questions per set
    itemsperq=design.shape[1]#items per questions
    itemsperset=np.max(design)#items per set
    people=np.random.choice([x for x in range(0,numreal)],nrespondents)#choice the bots
    cov=np.eye(nitems)
    totalrows=qperset*itemsperq#rows
    nb=np.zeros(nitems)
    nw=np.zeros(nitems)
    ss=samplescheme(nitems,k, batchsize, itemsperset)
    allquestions=np.zeros((nrespondents*qperset,itemsperq),dtype=np.int)
    allbest=np.zeros(nrespondents*qperset,dtype=np.int)
    allworst=np.zeros(nrespondents*qperset,dtype=np.int)
    X=np.zeros((nrespondents*qperset,nitems))
    w=np.zeros(nitems)
    numrows=nrespondents//batchsize
    betas=np.zeros((numrows,nitems))
    for iternum in range(0,nrespondents):
        util=utilsall[people[iternum],:]
        if mis>0 and iternum<=50:
            lowval=np.percentile(util,25)
            util[topmis]=lowval
        vernum=iternum%numver
        respdesign=design[qperset*vernum:(qperset)*(vernum+1)]-1
        questions=ss.get_questions(respdesign)#gets items
        bestitems,worstitems=fillsurvey(questions,util)
        for i,bitem in enumerate(bestitems):
            nb[bitem]+=1
            allbest[iternum*qperset+i]=bitem
        for i,witem in enumerate(worstitems):
            nw[witem]+=1
            allworst[iternum*qperset+i]=witem
        allquestions[iternum*qperset:(iternum+1)*qperset]=questions
        for i,oneq in enumerate(questions):
            X[iternum*qperset+i,oneq]=1
        if iternum%batchsize==(batchsize-1):
            #return X[:(iternum+1)*qperset]
            tempX=X[:(iternum+1)*qperset]
            tempq=allquestions[:(iternum+1)*qperset]
            tempbest=allbest[:(iternum+1)*qperset]
            tempworst=allworst[:(iternum+1)*qperset]
            if iternum!=(nrespondents-1):
                ss.update(tempX,tempq,tempbest,tempworst)
            w=estimateparams(tempX,tempq,nb,nw,[])
            betas[(iternum+1)//(batchsize)-1]=w
    #return X,allquestions,allbest,allworst
    return betas,allquestions,allbest,allworst,people

In [14]:
def fillsurvey(questions,util):
    qperset,itemsperq=questions.shape
    noiseb= -np.log(-np.log(np.random.rand(qperset,itemsperq)))
    noisew=  np.log(-np.log(np.random.rand(qperset,itemsperq)))
    combutilb=util[questions]+noiseb
    combutilw=util[questions]+noisew
    #indexb=np.argmax(combutilb,1)
    #indexw=np.argmin(combutilw,1)
    best=questions[range(qperset),np.argmax(combutilb,1)]
    worst=questions[range(qperset),np.argmin(combutilw,1)]
    return best,worst

In [41]:
def estimateparams(X,questions,nb,nw,w0=[],tol=.0001,maxiter=100,hessian=0):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems or (np.abs(w0)>4).any():
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    iternum=0
    inplay=np.where(np.bincount(questions.flatten(),minlength=nitems)>0)[0]
    notinplay=np.where(np.bincount(questions.flatten(),minlength=nitems)==0)[0]
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            gradb=nb-np.sum(Pb,0)
            gradw=-nw+np.sum(Pw,0)
            grad=-(gradb+gradw)
            Hb=np.dot(Pb.T,Pb)
            Hbd=-np.sum(Pb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(Pw.T,Pw)
            Hwd=-np.sum(Pw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[inplay[:-1]]
            Hl=H[inplay[:-1],:][:,inplay[:-1]]
            #Hlinv=np.linalg.pinv(Hl)
            #Hinvgradl=np.dot(Hlinv,gradl)
            Hinvgradl=np.linalg.solve(Hl,gradl)
            w[inplay[:-1]]=w[inplay[:-1]]-Hinvgradl
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except Exception:
        if hessian:
            return np.random.randn(nitems)/4,np.eye(nitems-1)
        else:
            return np.random.randn(nitems)/4
    if np.isnan(w).any():
        return np.random.randn(nitems)/4
        #raise Exception('w contains nan')
    w[notinplay]=w[np.random.choice(inplay,notinplay.shape[0])]+.001
    
    if hessian:
        H=np.eye(nitems-1)
        H[inplay[:-1],:][:,inplay[:-1]]=Hl
        return w,H
    else:
        return w

In [16]:
def bayesBootstrap(X,questions,allbest,allworst,numdraws=1):
    [numofq,nitems]=X.shape
    draws=np.zeros((numdraws,nitems))
    #w=estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=np.ones(numofq))
    for i in range(numdraws):
        #weights=np.random.dirichlet(np.ones(numofq))*numofq
        weights=np.random.exponential(1,numofq)
        draws[i]=estimateparamsbayes(X,questions,allbest,allworst,w0=[],weights=weights)
    return draws

In [17]:
def estimateparamsbayes(X,questions,allbest,allworst,w0=[],weights=[],tol=.0001,maxiter=100):
    
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems or (np.abs(w0)>4).any():
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    if len(weights)<numofq:
        weights=np.ones(numofq)
    iternum=0
    nb=np.bincount(allbest,weights,minlength=nitems)
    nw=np.bincount(allworst,weights,minlength=nitems)
    inplay=np.where(np.bincount(questions.flatten(),minlength=nitems)>0)[0]
    notinplay=np.where(np.bincount(questions.flatten(),minlength=nitems)==0)[0]
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            wPb=(Pb.T*weights).T
            wPw=(Pw.T*weights).T
            gradb=nb-np.sum(wPb,0)
            gradw=-nw+np.sum(wPw,0)
            grad=-(gradb+gradw)
            Hb=np.dot(wPb.T,Pb)
            Hbd=-np.sum(wPb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(wPw.T,Pw)
            Hwd=-np.sum(wPw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[inplay[:-1]]
            Hl=H[inplay[:-1],:][:,inplay[:-1]]
            
            #Hinvgradl=np.dot(np.linalg.pinv(Hl),gradl)
            Hinvgradl=np.linalg.solve(Hl,gradl)
            
            w[inplay[:-1]]=w[inplay[:-1]]-Hinvgradl
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        
        print(np.linalg.matrix_rank(Hl))
        #raise Exception('Not invertible')
        return np.random.randn(nitems)/4
    if np.isnan(w).any():
        raise Exception('w contains nan')
    w[notinplay]=w[np.random.choice(inplay,notinplay.shape[0])]+.001
    return w

In [18]:
def fixedexpress(nitems,N):
    return np.random.choice([x for x in range(0,nitems)],N,False)

In [19]:
def getX(numofq,nitems,allthem):
    X=np.zeros((numofq,nitems))
    X[range(numofq),allthem]=1
    return X

In [20]:
class TS:
    def __init__(self,nitems,k, bsize, itemsperset,esp=1/4,delta=.25,thres=0):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.nondif=itemsperset-int(np.round(itemsperset*esp))
        self.dif=int(np.round(itemsperset*esp))
        self.delta=delta
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.rowiter=0
        self.thres=thres
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.bsize,nitems))
        
        
        if self.thres:
            nb=np.bincount(allbest,minlength=nitems)
            nw=np.bincount(allworst,minlength=nitems)
            w=estimateparams(X,questions,nb,nw)
            order=np.argsort(w)[::-1]
            threshold=(w[order[self.k-1]]+w[order[self.k]])/2
        

        #w=estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=np.ones(numofq))
        for i in range(self.bsize):
            weights=np.random.exponential(1,numofq)
            draws[i]=estimateparamsbayes(X,questions,allbest,allworst,weights=weights)
        if self.thres:
            draws=-np.abs(threshold-draws)
        if self.dif:
            drawsdif=np.zeros((self.bsize,nitems))
            for i in range(self.bsize):
                subsamp=int(numofq*self.delta)
                sub=np.random.choice([x for x in range(0,numofq)],subsamp)
                tempX=X[sub]
                tempb=allbest[sub]
                tempw=allworst[sub]
                tempq=questions[sub]
                weights=np.random.exponential(1,subsamp)
                drawsdif[i]=estimateparamsbayes(tempX,tempq,tempb,tempw,weights=weights)
            if self.thres:
                drawsdif=-np.abs(threshold-drawsdif)
            
        for i in range(self.bsize):
            if self.dif:
                start=np.argsort(draws[i])[::-1][:self.nondif]
                difsort=np.argsort(drawsdif[i])[::-1]
                diffuse=[x for x in difsort if x not in start]
                self.items[i,:self.nondif]=start
                self.items[i,self.nondif:]=diffuse[:self.dif]
            else:
                start=np.argsort(draws[i])[::-1][:self.itemsperset]
                self.items[i,:]=start
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [48]:
class TSapprox:
    def __init__(self,nitems,k, bsize, itemsperset,esp=1/4,delta=10,thres=0):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.nondif=itemsperset-int(np.round(itemsperset*esp))
        self.dif=int(np.round(itemsperset*esp))
        self.delta=delta
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.rowiter=0
        self.thres=thres
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.bsize,nitems))
        
        nb=np.bincount(allbest,minlength=nitems)
        nw=np.bincount(allworst,minlength=nitems)
        w,H=estimateparams(X,questions,nb,nw,hessian=1)
        cov=np.linalg.inv(H)
        draws[:,:-1]=np.random.multivariate_normal(w[:-1], cov,self.bsize)
        
        if self.thres:
            order=np.argsort(draws)[:,::-1]
            threshold=(draws[range(self.bsize),order[:,self.k-1]]+draws[range(self.bsize),order[:,self.k]])/2
            draws=-np.abs(threshold-draws.T).T

        #w=estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=np.ones(numofq))
        if self.dif:
            drawsdif=np.zeros((self.bsize,nitems))
            drawsdif[:,:-1]=np.random.multivariate_normal(w[:-1], self.delta*cov,self.bsize)
            if self.thres:
                order=np.argsort(drawsdif)[:,::-1]
                threshold=(drawsdif[range(self.bsize),order[:,self.k-1]]+drawsdif[range(self.bsize),order[:,self.k]])/2
                drawsdif=-np.abs(threshold-drawsdif.T).T
            
        for i in range(self.bsize):
            if self.dif:
                start=np.argsort(draws[i])[::-1][:self.nondif]
                difsort=np.argsort(drawsdif[i])[::-1]
                diffuse=[x for x in difsort if x not in start]
                self.items[i,:self.nondif]=start
                self.items[i,self.nondif:]=diffuse[:self.dif]
            else:
                start=np.argsort(draws[i])[::-1][:self.itemsperset]
                self.items[i,:]=start
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [22]:
class TSkdouble:
    def __init__(self,nitems,k, bsize, itemsperset,esp=1/4,delta=.25):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.nondif=k-int(np.round(k*esp))
        self.dif=int(np.round(k*esp))
        self.delta=delta
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.rowiter=0
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.bsize//2,nitems))
        
        

        #w=estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=np.ones(numofq))
        for i in range(self.bsize//2):
            weights=np.random.exponential(1,numofq)
            draws[i]=estimateparamsbayes(X,questions,allbest,allworst,weights=weights)

        if self.dif:
            drawsdif=np.zeros((self.bsize//2,nitems))
            for i in range(self.bsize//2):
                subsamp=int(numofq*self.delta)
                sub=np.random.choice([x for x in range(0,numofq)],subsamp)
                tempX=X[sub]
                tempb=allbest[sub]
                tempw=allworst[sub]
                tempq=questions[sub]
                weights=np.random.exponential(1,subsamp)
                drawsdif[i]=estimateparamsbayes(tempX,tempq,tempb,tempw,weights=weights)

            
        for i in range(self.bsize//2):
            if self.dif:
                start=np.argsort(draws[i])[::-1][:self.nondif]
                difsort=np.argsort(drawsdif[i])[::-1]
                diffuse=[x for x in difsort if x not in start]
                self.items[2*i,:self.nondif//2]=start[::2]
                self.items[2*i,self.nondif//2:]=diffuse[:self.dif:2]
                self.items[2*i+1,:self.nondif//2]=start[1::2]
                self.items[2*i+1,self.nondif//2:]=diffuse[1:self.dif+1:2]
            else:
                start=np.argsort(draws[i])[::-1][:self.itemsperset*2]
                self.items[2*i,:]=start[::2]
                self.items[2*i+1,:]=start[1::2]
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [23]:
class greedy:
    def __init__(self,nitems,k, bsize, itemsperset,esp=1/4,thres=0):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.nondif=itemsperset-int(np.round(itemsperset*esp))
        self.dif=int(np.round(itemsperset*esp))
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.rowiter=0
        self.thres=thres
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.bsize,nitems))
        nb=np.bincount(allbest,minlength=nitems)
        nw=np.bincount(allworst,minlength=nitems)
        w=estimateparams(X,questions,nb,nw)
        if self.thres:
            order=np.argsort(w)[::-1]
            threshold=(w[order[self.k-1]]+w[order[self.k]])/2
            start=np.argsort(-np.abs(threshold-w))[::-1][:self.nondif]
        else:
            start=np.argsort(w)[::-1][:self.nondif]
        
        
        drawsdif=np.zeros((self.bsize,self.itemsperset))
        for i in range(self.bsize):
            drawsdif[i]=fixedexpress(nitems,self.itemsperset)
            
        for i in range(self.bsize):
            difsort=drawsdif[i]
            diffuse=[x for x in difsort if x not in start]
            self.items[i,:self.nondif]=start
            self.items[i,self.nondif:]=diffuse[:self.dif]
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [24]:
class greedythres:
    def __init__(self,nitems,k, bsize, itemsperset,esp=1/4):
        self.greed=greedy(nitems,k, bsize, itemsperset,esp,1)
        
    def update(self,X,questions,allbest,allworst):
        self.greed.update(X,questions,allbest,allworst)

    
    def get_questions(self,design):
        return self.greed.get_questions(design)

In [ ]:
class greedynightmare:
    def __init__(self,nitems,k, bsize, itemsperset,esp=1/4,thres=0):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.nondif=itemsperset-int(np.round(itemsperset*esp))
        self.dif=int(np.round(itemsperset*esp))
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.rowiter=0
        self.thres=thres
        self.iteration=1
        self.w=np.zeros(nitems)
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.bsize,nitems))
        
        nb=np.bincount(allbest[-self.bsize:],minlength=nitems)
        nw=np.bincount(allworst[-self.bsize:],minlength=nitems)
        zb=np.sum(np.exp(w[questions[-self.bsize:]]),1)
        zw=np.sum(np.exp(-w[questions[-self.bsize:]]),1)
        Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
        Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
        gradb=nb-np.sum(Pb,0)
        gradw=-nw+np.sum(Pw,0)
        grad=-(gradb+gradw)
        w[:-1]=w[:-1]-1/self.iteration*grad[:-1]
        self.iteration+=1
        if self.thres:
            order=np.argsort(w)[::-1]
            threshold=(w[order[self.k-1]]+w[order[self.k]])/2
            start=np.argsort(-np.abs(threshold-w))[::-1][:self.nondif]
        else:
            start=np.argsort(w)[::-1][:self.nondif]
        
        
        drawsdif=np.zeros((self.bsize,self.itemsperset))
        for i in range(self.bsize):
            drawsdif[i]=fixedexpress(nitems,self.itemsperset)
            
        for i in range(self.bsize):
            difsort=drawsdif[i]
            diffuse=[x for x in difsort if x not in start]
            self.items[i,:self.nondif]=start
            self.items[i,self.nondif:]=diffuse[:self.dif]
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [25]:
class TSthres:
    def __init__(self,nitems,k, bsize, itemsperset):
        self.ts=TS(nitems,k, bsize, itemsperset,thres=1)
        
    def update(self,X,questions,allbest,allworst):
        self.ts.update(X,questions,allbest,allworst)

    
    def get_questions(self,design):
        return self.ts.get_questions(design)

In [26]:
class TSapproxthres:
    def __init__(self,nitems,k, bsize, itemsperset):
        self.ts=TSapprox(nitems,k, bsize, itemsperset,thres=1)
        
    def update(self,X,questions,allbest,allworst):
        self.ts.update(X,questions,allbest,allworst)

    
    def get_questions(self,design):
        return self.ts.get_questions(design)

In [27]:
class TSregthres:
    def __init__(self,nitems,k, bsize, itemsperset):
        self.ts=TS(nitems,k, bsize, itemsperset,esp=0,thres=1)
        
    def update(self,X,questions,allbest,allworst):
        self.ts.update(X,questions,allbest,allworst)

    def get_questions(self,design):
        return self.ts.get_questions(design)

In [28]:
class TSreg:
    def __init__(self,nitems,k, bsize, itemsperset):
        self.ts=TS(nitems,k, bsize, itemsperset,esp=0)
        
    def update(self,X,questions,allbest,allworst):
        self.ts.update(X,questions,allbest,allworst)

    
    def get_questions(self,design):
        return self.ts.get_questions(design)

In [29]:
class misclassMinapprox:
    def __init__(self,nitems,k, bsize, itemsperset,N=100):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.N=N
        self.rowiter=0
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.N,nitems))
        nb=np.bincount(allbest,minlength=nitems)
        nw=np.bincount(allworst,minlength=nitems)
        
        w,H=estimateparams(X,questions,nb,nw,hessian=1)
        cov=np.linalg.inv(H)
        draws[:,:-1]=np.random.multivariate_normal(w[:-1], cov,self.N)
        

        score=np.bincount(np.argsort(draws,1)[:,::-1][:,:self.k].flatten(),
                          minlength=nitems)/self.N
        order=np.argsort(w)[::-1]
        score[order[:self.k]]=1-score[order[:self.k]]
        var=.1/(np.bincount(questions.flatten(),minlength=nitems)+.01)
        zeros=np.zeros(nitems)
        
        for i in range(self.bsize):
            newscore=score+np.random.normal(zeros,np.sqrt(var))
            start=np.argsort(newscore)[::-1][:self.itemsperset]
            self.items[i]=start
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [30]:
class misclassMin:
    def __init__(self,nitems,k, bsize, itemsperset,N=100):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.N=N
        self.rowiter=0
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.N,nitems))
        nb=np.bincount(allbest,minlength=nitems)
        nw=np.bincount(allworst,minlength=nitems)
        w=estimateparams(X,questions,nb,nw)
        
        for i in range(self.N):
            weights=np.random.exponential(1,numofq)
            draws[i]=estimateparamsbayes(X,questions,allbest,allworst,weights=weights)
        score=np.bincount(np.argsort(draws,1)[:,::-1][:,:self.k].flatten(),
                          minlength=nitems)/self.N
        order=np.argsort(w)[::-1]
        score[order[:self.k]]=1-score[order[:self.k]]
        var=.1/np.bincount(questions.flatten(),minlength=nitems)
        zeros=np.zeros(nitems)
        
        for i in range(self.bsize):
            newscore=score+np.random.normal(zeros,np.sqrt(var))
            start=np.argsort(newscore)[::-1][:self.itemsperset]
            self.items[i]=start
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        return items[design]

In [31]:
class greatUncert:
    def __init__(self,nitems,k, bsize, itemsperset,N=100):
        self.bsize = bsize
        self.k=k
        self.itemsperset = itemsperset
        self.items=np.zeros((bsize,itemsperset),dtype=np.int)
        self.N=N
        self.rowiter=0
        for i in range(bsize):
            self.items[i]=fixedexpress(nitems,itemsperset)
        
    def update(self,X,questions,allbest,allworst):
        self.rowiter=0
        [numofq,nitems]=X.shape
        draws=np.zeros((self.N,nitems))
        
        for i in range(self.N):
            weights=np.random.exponential(1,numofq)
            draws[i]=estimateparamsbayes(X,questions,allbest,allworst,weights=weights)
        score=np.bincount(np.argsort(draws,1)[:,::-1][:,:self.k].flatten(),
                          minlength=nitems)/self.N
        score=-np.abs(.5-score)
        
        var=.1/np.bincount(questions.flatten(),minlength=nitems)
        zeros=np.zeros(nitems)
        for i in range(self.bsize):
            newscore=score+np.random.normal(zeros,np.sqrt(var))
            start=np.argsort(newscore)[::-1][:self.itemsperset]
            self.items[i]=start
    
    def get_questions(self,design):
        items=self.items[self.rowiter]
        self.rowiter+=1
        print(items)
        return items[design]

In [32]:
def run100trials(samplescheme,filename,k=10,itemnum=120,dnum=30):
    if itemnum==300:
        itemfile='HB_300items.csv'
        nitems=300
    else:
        itemfile='HB_120items.csv'
        nitems=120
    if dnum==20:
        designfile='20itemMD_MD20_Design.csv'
        nofqsbot=12
    else:
        designfile='30itemMD_MD30_Design.csv'
        nofqsbot=18
    n=100
    numresp=500
    #fix not manuel
    choicedata=np.zeros((n*numresp*nofqsbot,11),dtype=np.int)
    batchsize=20
    numrows=numresp//batchsize
    allbetas=np.zeros((numrows*n,nitems+2))
    for i in range(n):
        betas,allquestions,allbest,allworst,people=simulation(
            numresp,k,samplescheme,batchsize,itemfile,designfile,0)
        rowstart=i*numresp*nofqsbot
        rowend=(i+1)*numresp*nofqsbot
        choicedata[rowstart:rowend,0]=i+1
        for j,person in enumerate(people):
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,1]=person
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,2]=j
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,3]=[x for x in range(1,nofqsbot+1)]
        choicedata[rowstart:rowend,4:9]=allquestions+1
        choicedata[rowstart:rowend,9]=allbest+1
        choicedata[rowstart:rowend,10]=allworst+1
        allbetas[i*numrows:(i+1)*numrows,0]=i+1
        allbetas[i*numrows:(i+1)*numrows,1]=[x for x in range(20,numrows*20+20,20)]
        allbetas[i*numrows:(i+1)*numrows,2:]=betas
    choicedata_df=pd.DataFrame(choicedata)
    choicedata_df.columns=["Iter","original_ids","RespIDNum","QNum",
                           "Choice1","Choice2","Choice3","Choice4","Choice5",
                           "Best","Worst"]
    allbetas_df=pd.DataFrame(allbetas)
    colname=["Iter","RespNum"]
    for i in range(nitems):
        colname+=["Item_"+str(i+1)]
    allbetas_df.columns=colname
    choicedata_df.to_csv("Results/choice_data-"+filename)
    allbetas_df.to_csv("Results/bhist-"+filename)

In [33]:
k=10

In [49]:
t=time.time()
betas,allquestions,allbest,allworst,people=simulation(500,k,
                                                      TSapproxthres,20,'HB_120items.csv','20itemMD_MD20_Design.csv',0,20)
time.time()-t

9.846861124038696

In [251]:
t=time.time()
betas2,allquestions,allbest,allworst,people=simulation(500,k,
                                                      TSthres,20,'HB_120items.csv','20itemMD_MD20_Design.csv',0,20)
time.time()-t

109.7612829208374

In [211]:
resp=100
nitems=120
subsamp=int(12*resp*.25)
sub=np.random.choice([x for x in range(0,12*resp)],subsamp)

tempq=allquestions[sub]

In [212]:
notin=np.where(np.bincount(tempq.flatten(),minlength=nitems)==0)[0]

In [225]:
notin

array([55, 81])

In [214]:
ins=np.where(np.bincount(tempq.flatten(),minlength=nitems)>0)[0]

In [215]:
H=np.random.rand(120,120)

In [216]:
H[ins,:][:,ins].shape

(118, 118)

In [239]:
ind=np.random.choice(ins,notin.shape[0])

In [240]:
ind

array([74, 39])

In [241]:
betas[3,notin]=betas[3,ind]+.001

In [242]:
np.argsort(betas[3])

array([  2,  80,  57,   0,  35,  53,  24,  49,  40, 102,  45,  46,  98,
       113,   8, 117,  95,  33,  15,  18,  79, 111, 116, 101,  84,  12,
       100,   6,  76,  10,  42,  54,  89,  93,  17, 115,  75, 118, 104,
        36,  11,  52,  87,  59,  63,  23,  99,  64,   3,  71,  30,  66,
         1,   7, 112, 109,  72,  92,  85, 114,  86,  61,  16, 103, 108,
        27,  44, 105,  97,  65,  56,  21,  13,  62,  96, 106,  14,  68,
        58,  39,  81, 107,  31,  50,  69,   4, 119,  26,  22,  82,  34,
        78,  94,   9,  90,  32,  37,  83,  43,  47,  51,  77,  67,  48,
        19,  74,  55,  91,  73,  60,  70,  20,  29,  28,  41,  88,  38,
        25,   5, 110])

In [248]:
k=10
data1=pd.read_csv('HB_120items.csv')
utilsall=data1.get_values()[:,2:]
realmean=np.mean(utilsall,0)
topk=np.argsort(realmean)[::-1][:k]

In [249]:
topk

array([ 72,  62,  48, 105, 104, 112,  71,  54,  36,  53])

In [40]:
allquestions[5]

array([108,  59,  43,  70,  45])

In [255]:
[num,numitems]=betas.shape
topresp=np.zeros(num)
resp=[x for x in range(20,520,20)]
for i,cur in enumerate(np.argsort(betas)[:,::-1][:,:k]):
        topresp[i]=(len(set.intersection(set(cur),set(topk))))
topresp2=np.zeros(num)
resp=[x for x in range(20,520,20)]
for i,cur in enumerate(np.argsort(betas2)[:,::-1][:,:k]):
        topresp2[i]=(len(set.intersection(set(cur),set(topk))))

In [256]:
import matplotlib.pyplot as plt
plt.plot(resp,topresp,resp,topresp2)
plt.show()

In [121]:
filename="TSdouble-120v20k40.csv"
run100trials(TSkdouble,filename,k=40,itemnum=120,dnum=20)

In [128]:
itemcol=[]
for i in range(1,121):
    itemcol+=['Item_'+str(i)]

In [130]:
resps=50
iters=100
topresp=np.zeros((iters,resps))

In [140]:
#filename='fixed_express-120v20.csv'
filename="TSdouble-120v20k40.csv"
data=pd.read_csv('Results/bhist-'+filename)
for j in range(100):

    iterdata=data[data['Iter']==(j+1)]
    for i,cur in enumerate(np.argsort(iterdata[itemcol].get_values())[:,::-1][:,:k]):
        topresp[j,i]=(len(set.intersection(set(cur),set(topk))))
topkmean1=np.mean(topresp,0)

In [141]:
import matplotlib.pyplot as plt
plt.plot(resp,topkmean[:25])
plt.plot(resp,topkmean1[:25])
plt.show()